In [1]:
# 신경망을 학습할 때 가장 자주 사용되는 알고리즘 : 역전파
# 매개변수 : 주어진 매개변수에 대한 손실함수의 변화도 (gradient)에 따라 조정됨
# 이러한 변화도를 계산하기 위해 torch.autograd 라는 자동 미분 엔진 존재

In [2]:
import torch

x = torch.ones(5) # input tensor
y = torch.zeros(3) # expected tensor
w = torch.randn(5, 3, requires_grad = True)
b = torch.randn(3, requires_grad = True)
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [8]:
print("Gradient function for z =", z.grad_fn)
print("Gradient function for loss =", loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7fc860fa11c0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7fc860fa10d0>


In [9]:
# 변화도 계산하기
# 신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 손실함수의 도함수를 계산해야 함
# 도함수를 계산하기 위해, loss.backward()를 호출한 다음, w.grad / b.grad에서 값을 가져옴

loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2084, 0.0844, 0.0323],
        [0.2084, 0.0844, 0.0323],
        [0.2084, 0.0844, 0.0323],
        [0.2084, 0.0844, 0.0323],
        [0.2084, 0.0844, 0.0323]])
tensor([0.2084, 0.0844, 0.0323])


In [10]:
# 변화도 추적 멈추기
# requires_grad = True인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원함
# 모델을 학습한 뒤, 입력 데이터를 단순히 적용하기만 하는 경우 (순전파만 필요한 경우) -> 이러한 추적이나 지원이 필요 없을 수 있음
# torch.no_grad() 블록으로 연산 코드를 감싸서 연산 추적을 멈출 수 있음

z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b

print(z.requires_grad)

True
False


In [11]:
# detach() 메소드를 사용해서 연산 추적을 멈출수도 있음

z = torch.matmul(x, w) + b
z_det = z.detach()
print(z_det.requires_grad)

False


In [ ]:
## 변화도 추적을 멈춰야하는 이유
### 신경망의 일부 매개변수를 고정된 매개변수 (frozen parameter)로 표시함. 이는 사전학습된 신경망을 미세조정할 때 매우 일반적임
### 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, 순전파 단계만 수행할 때 연산 속도가 향상됨

In [ ]:
# 개념적으로, autograd는 실행된 모든 연산들의 기록을 Function 객체로 구성된 방향성 비순환 그래프(DAG)에 저장함
# 이 DAG의 잎은 입력 텐서이고, 뿌리는 결과 텐서이다.
# DAG를 뿌리에서 잎까지 추적하면서 연쇄법칙에 따라 변화도를 자동으로 계산할 수 있다.

# 순전파 단계
### 요청된 연산을 수행하여 결과 텐서를 계산함
### DAG에 연산의 gradient function을 유지함

# 역전파 단계
### 각 .grad_fn으로부터 변화도를 계산하고
### 각 텐서의 .grad 속성에 계산 결과를 쌓고
### 연쇄법칙을 사용하여, 모든 잎 텐서들까지 전파함

In [4]:
# 대부분의 경우, 스칼라 손실 함수를 가지고 일부 매개변수와 관련된 변화도를 계산해야 함
# 그러나, 출력 함수가 임의의 텐서인 경우 실제 변화도가 아닌 야코비안 곱을 계산함

inp = torch.eye(5, requires_grad = True)
out = (inp + 1).pow(2)
out.backward(torch.ones_like(inp), retain_graph = True)
print('First call\n', inp.grad)

out.backward(torch.ones_like(inp), retain_graph = True)
print('\nSecond call\n',inp.grad)

inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph = True)
print('\nCall after zeroing gradients\n', inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


In [ ]:
# 동일한 인자로 backward를 두 차례 호출하면 변화도 값이 달라짐
# 이는 역방향 전파를 수행할 때, pytorch가 변화도를 누적해두기 때문
# 계산된 변화도의 값이 연산 그래프의 모든 잎 노드의 grad 속성에 추가됨 -> 제대로 된 변화도를 계산하기 위해서는 grad 속성을 먼저 0으로 만들어야 함
# 실제 학습 과정에서는 optimizer가 이 과정을 도와줌